<a href="https://colab.research.google.com/github/MatteoOnger/NIAC_Project/blob/dev/NIAC_PMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NIAC Project: Pacman Maze Solver**

*   **Authors:** Stefano Capelli, Matteo Onger
*   **Date:** March 2025

**Documentation**:
*   Gymnasium: [website](https://gymnasium.farama.org/)
*   Scallop & Scallopy: [paper](https://www.researchgate.net/publication/369945806_Scallop_A_Language_for_Neurosymbolic_Programming), [repository](https://github.com/scallop-lang/scallop), [website](https://www.scallop-lang.org/)
*   Pacman Maze game: [description](https://www.scallop-lang.org/ssnp24/index.html#section-17:~:text=2%3A%20PacMan%20Agent-,In,-this%20part%2C%20we)

**Notes**:
*   ...

In [ ]:
# download project repository
!git clone -b dev https://github.com/MatteoOnger/NIAC_Project.git

# download Miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-py311_25.1.1-2-Linux-x86_64.sh

# download Scallopy 0.2.4 (Python 3.10)
!wget https://github.com/scallop-lang/scallop/releases/download/0.2.4/scallopy-0.2.4-cp310-cp310-manylinux_2_27_x86_64.whl

In [ ]:
# install Miniconda
%env PYTHONPATH=

!chmod +x /content/Miniconda3-py311_25.1.1-2-Linux-x86_64.sh
!bash Miniconda3-py311_25.1.1-2-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python3.11/site-packages/')

In [ ]:
# force creation of the virtual environment 'niac'
!conda env remove -n niac -y || true
!conda env create -f /content/NIAC_Project/environment.yaml -y
!conda env list

---
## Installation check

In [ ]:
# hello world in Scallopy 0.2.4
%%bash
source activate niac
python3

import sys
print(f"-----\nCurrent Python version: {sys.version}\n-----")

import scallopy
ctx = scallopy.ScallopContext()

ctx.add_relation("hello", str)
ctx.add_facts("hello", [("Hello World",)])
ctx.run()

print(list(ctx.relation("hello")))
quit()

In [ ]:
# random agent plays pacman maze
import matplotlib.pyplot as plt
import numpy as np
import time

from IPython.display import clear_output
from NIAC_Project.pacman.arena import AvoidingArena

env = AvoidingArena(render_mode="rgb_array", grid_dim=(4,4), num_enemies=3)
env.reset()

counter = 0
tot_reward = 0
end_episode = False

while not end_episode:
    action = np.random.randint(0, 4)
    observation, reward, terminated, truncated, info = env.step(action)
    tot_reward += reward


    clear_output(wait=True)
    plt.imshow(env.render())
    plt.axis('off')
    plt.show()
    time.sleep(.1)

    end_episode = terminated or truncated or (counter > 10)
    counter += 1

env.close()
print(f"Tot reward:{tot_reward}")

---
## .......

In [28]:
%%bash
source activate niac
python3

import scallopy
import torch
from enum import Enum


class Actions(Enum):
    RIGHT = 0
    UP = 1
    LEFT = 2
    DOWN = 3

X, Y, eps = 3, 3, 0.1
nodes = [(i,j) for i in range(X) for j in range(Y)]

agent_p = torch.zeros(9)
target_p = torch.zeros(9)
enemy_p = torch.zeros(9)

agent_p[3] = 1.
target_p[8] = 1.
enemy_p[4] = 1.


# difftopkproofs, diffaddmultprob2
ctx = scallopy.ScallopContext(provenance="difftopkproofs")

ctx.add_relation("actor", (int, int))
ctx.add_relation("target", (int, int))
ctx.add_relation("node", (int, int))

ctx.add_facts("actor", [(agent_p[i], node) for i, node in enumerate(nodes)])
ctx.add_facts("target", [(target_p[i], node) for i, node in enumerate(nodes)])
ctx.add_facts(
    "node",
    [(torch.clip(1 - enemy_p[i] - eps, min=0), node) for i, node in enumerate(nodes)],
)

ctx.add_rule(f"edge(x, y, xp, y, {Actions.RIGHT.value}) = node(x, y) and node(xp, y) and xp == x + 1")
ctx.add_rule(f"edge(x, y, x, yp, {Actions.UP.value})    = node(x, y) and node(x, yp) and yp == y + 1")
ctx.add_rule(f"edge(x, y, xp, y, {Actions.LEFT.value})  = node(x, y) and node(xp, y) and xp == x - 1")
ctx.add_rule(f"edge(x, y, x, yp, {Actions.DOWN.value})  = node(x, y) and node(x, yp) and yp == y - 1")

ctx.add_rule("path(x, y, x, y) = node(x, y)")
ctx.add_rule("path(x, y, xp, yp) = edge(x, y, xp, yp, _)")
ctx.add_rule("path(x, y, xpp, ypp) = path(x, y, xp, yp) and edge(xp, yp, xpp, ypp, _)")

ctx.add_rule("next_position(a, xp, yp) = actor(x, y) and edge(x, y, xp, yp, a)")
ctx.add_rule("next_action(a) = next_position(a, x, y) and path(x, y, gx, gy) and target(gx, gy)")

ctx.run()

agents = list(ctx.relation("actor"))
targets = list(ctx.relation("target"))
nodes = list(ctx.relation("node"))
edges = list(ctx.relation("edge"))
paths = list(ctx.relation("path"))
next_positions = list(ctx.relation("next_position"))
next_actions = list(ctx.relation("next_action"))

for i, node in enumerate(nodes):
    print(f"{node[1]} -> {node[0]}, agents={agents[i][0]},  targets={targets[i][0]}, enemy={enemy_p[i]}")
print("-" * 50)

for next_position in next_positions:
    if next_position[0] != 0:
        print(f"{next_position[1]} -> {next_position[0]}")
print("-" * 50)

print(next_actions)
quit()

(0, 0) -> 0.8999999761581421, agents=0.0,  targets=0.0, enemy=0.0
(0, 1) -> 0.8999999761581421, agents=0.0,  targets=0.0, enemy=0.0
(0, 2) -> 0.8999999761581421, agents=0.0,  targets=0.0, enemy=0.0
(1, 0) -> 0.8999999761581421, agents=1.0,  targets=0.0, enemy=0.0
(1, 1) -> 0.0, agents=0.0,  targets=0.0, enemy=1.0
(1, 2) -> 0.8999999761581421, agents=0.0,  targets=0.0, enemy=0.0
(2, 0) -> 0.8999999761581421, agents=0.0,  targets=0.0, enemy=0.0
(2, 1) -> 0.8999999761581421, agents=0.0,  targets=0.0, enemy=0.0
(2, 2) -> 0.8999999761581421, agents=0.0,  targets=1.0, enemy=0.0
--------------------------------------------------
(0, 2, 0) -> 0.809999942779541
(2, 0, 0) -> 0.809999942779541
--------------------------------------------------
[(tensor(0.6561, requires_grad=True), (0,)), (tensor(0., requires_grad=True), (1,)), (tensor(0.6915, requires_grad=True), (2,)), (tensor(0., requires_grad=True), (3,))]
